In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from torchvision import models, datasets, transforms
import time
import numpy as np
from tqdm import tqdm

In [2]:
# Define data augmentations for training
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [1]:
pip install codecarbon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.2/504.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.4/386.4 kB 24.9 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0
  Attempting uninstall: prompt_toolkit
    Found existing installation: prompt_toolkit 3.0.48
    Uninstalling prompt_toolkit-3.0.48:
      Successfully uninstalled prompt_toolkit-3.0.48
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
i

In [3]:
pip install thop

In [4]:
pip install ptflops

In [5]:
# Load CIFAR-10 dataset with train/test splits
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split the training data into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4, pin_memory=True)


100%|██████████| 170M/170M [00:03<00:00, 48.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [6]:
# Load the ResNet-50 model and modify the final layer
teacher_model = models.resnet50(pretrained=True)
teacher_model.fc = nn.Linear(2048, 10)  # CIFAR-10 has 10 classes

# Check if GPU is available and move model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model.to(device)

# Define loss function, optimizer, and learning rate scheduler
criterion_ce = nn.CrossEntropyLoss()
optimizer_teacher = optim.Adam(teacher_model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = StepLR(optimizer_teacher, step_size=10, gamma=0.1)  # Reduce LR by 0.1 every 10 epochs

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 177MB/s]


In [7]:
# Training loop
num_epochs_teacher = 30  # Increased epochs for better accuracy
best_val_accuracy = 0.0
start_time_teacher = time.time()

for epoch in range(num_epochs_teacher):
    # Training phase
    teacher_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in tqdm(train_loader, desc=f'Teacher Model Training Epoch [{epoch + 1}/{num_epochs_teacher}]'):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer_teacher.zero_grad()
        outputs = teacher_model(inputs)
        loss = criterion_ce(outputs, labels)
        loss.backward()
        optimizer_teacher.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Validation phase
    teacher_model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = teacher_model(inputs)
            loss = criterion_ce(outputs, labels)
            running_val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = running_val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    # Save the model if validation accuracy improves
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(teacher_model.state_dict(), 'teacher_model_best.pth')
        print(f"Best model saved with accuracy: {best_val_accuracy:.2f}%")

    # Step the learning rate scheduler
    scheduler.step()

    print(f"Epoch [{epoch + 1}/{num_epochs_teacher}], "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

end_time_teacher = time.time()
print(f"Teacher training completed in {(end_time_teacher - start_time_teacher) / 60:.2f} minutes")


Teacher Model Training Epoch [1/30]: 100%|██████████| 313/313 [00:45<00:00,  6.91it/s]


Best model saved with accuracy: 53.67%
Epoch [1/30], Train Loss: 1.3850, Train Accuracy: 51.98%, Val Loss: 1.3176, Val Accuracy: 53.67%


Teacher Model Training Epoch [2/30]: 100%|██████████| 313/313 [00:42<00:00,  7.30it/s]


Best model saved with accuracy: 64.93%
Epoch [2/30], Train Loss: 1.0584, Train Accuracy: 63.58%, Val Loss: 1.0253, Val Accuracy: 64.93%


Teacher Model Training Epoch [3/30]: 100%|██████████| 313/313 [00:42<00:00,  7.38it/s]


...

Best model saved with accuracy: 83.04%
Epoch [28/30], Train Loss: 0.3882, Train Accuracy: 86.56%, Val Loss: 0.4920, Val Accuracy: 83.04%


Teacher Model Training Epoch [29/30]: 100%|██████████| 313/313 [00:42<00:00,  7.40it/s]


Best model saved with accuracy: 83.39%
Epoch [29/30], Train Loss: 0.3914, Train Accuracy: 86.47%, Val Loss: 0.4913, Val Accuracy: 83.39%


Teacher Model Training Epoch [30/30]: 100%|██████████| 313/313 [00:42<00:00,  7.41it/s]


Epoch [30/30], Train Loss: 0.3914, Train Accuracy: 86.41%, Val Loss: 0.5053, Val Accuracy: 82.83%
Teacher training completed in 26.09 minutes


In [8]:
# # Load the best model for testing
# teacher_model.load_state_dict(torch.load('teacher_model_best_pruned.pth'))

# Testing phase
teacher_model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = teacher_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f'Accuracy of the teacher model on the test set: {test_accuracy:.2f}%')


Accuracy of the teacher model on the test set: 86.17%


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from torchvision import models, datasets, transforms
import torch.nn.utils.prune as prune
from ptflops import get_model_complexity_info
import numpy as np
from tqdm import tqdm
import time
import os
import codecarbon

# Initialize CodeCarbon tracker
tracker = codecarbon.EmissionsTracker()
tracker.start()

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define dataset transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 dataset
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split the dataset
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Load ResNet-50 model and modify for CIFAR-10
teacher_model = models.resnet50(pretrained=True)
teacher_model.fc = nn.Linear(2048, 10)  # CIFAR-10 has 10 classes
teacher_model.to(device)

# Define loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(teacher_model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

# Function to apply L1 unstructured pruning
def apply_pruning(model, amount=0.85):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)

# Apply pruning
apply_pruning(teacher_model, amount=0.85)

# Function to remove pruning and make it permanent
def remove_pruning(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            if hasattr(module, 'weight_mask'):
                prune.remove(module, 'weight')
    return model

# Measure FLOPs
def calculate_flops(model):
    model.eval()
    flops, params = get_model_complexity_info(model, (3, 32, 32), as_strings=False, print_per_layer_stat=False)
    return flops, params

# Count non-zero parameters
def count_non_zero_params(model):
    non_zero_count = 0
    for param in model.parameters():
        if param.requires_grad:
            non_zero_count += torch.count_nonzero(param).item()
    return non_zero_count

# Measure inference time
def measure_inference_time(model, dataloader):
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        for images, _ in dataloader:
            images = images.to(device)
            _ = model(images)
    end_time = time.time()
    return (end_time - start_time) / len(dataloader)

# Training loop
num_epochs_teacher = 10
best_val_accuracy = 0.0
start_time_teacher = time.time()

for epoch in range(num_epochs_teacher):
    # Training phase
    teacher_model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs_teacher} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = teacher_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Validation phase
    teacher_model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = teacher_model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = running_val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    # Save the best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(teacher_model.state_dict(), 'resnet50_pruned_best.pth')

    scheduler.step()

    print(f"Epoch [{epoch + 1}/{num_epochs_teacher}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

end_time_teacher = time.time()
training_time = (end_time_teacher - start_time_teacher) / 60

# Remove pruning and make it permanent
teacher_model = remove_pruning(teacher_model)

# Calculate metrics after pruning
flops, params = calculate_flops(teacher_model)
param_count = count_non_zero_params(teacher_model)
inference_time = measure_inference_time(teacher_model, test_loader)

# Stop CodeCarbon tracker
emissions = tracker.stop()

# Save the pruned model
torch.save(teacher_model.state_dict(), 'resnet50_pruned_final.pth')
model_size = os.path.getsize('resnet50_pruned_final.pth') / (1024 * 1024)

# Print final results
print("\n--- Model Analysis After Pruning ---")
print(f"Non-zero Parameter Count: {param_count}")
print(f"Model Size: {model_size:.2f} MB")
print(f"Training Time: {training_time:.2f} minutes")
print(f"Inference Time per Image: {inference_time:.6f} seconds")
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"CO2 Emissions: {emissions:.6f} kg")


[codecarbon INFO @ 23:36:17] [setup] RAM Tracking...
[codecarbon INFO @ 23:36:17] [setup] GPU Tracking...
[codecarbon INFO @ 23:36:17] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 23:36:17] [setup] CPU Tracking...
[codecarbon WARNING @ 23:36:17] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon WARNING @ 23:36:18] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon INFO @ 23:36:18] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 23:36:18] >>> Tracker's metadata:
[codecarbon INFO @ 23:36:18]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 23:36:18]   Python version: 3.10.12
[codecarbon INFO @ 23:36:18]   CodeCarbon version: 2.7.4
[codecarbon INFO @ 23:36:18]   Available RAM : 12.675 GB
[codecarbon INFO @ 23:36:18]   CPU count: 2
[codeca

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNe

Epoch [1/10], Train Loss: 0.9772, Train Accuracy: 66.13%, Val Loss: 0.6473, Val Accuracy: 78.10%


Epoch 2/10 - Training:  23%|██▎       | 145/625 [00:09<00:34, 13.77it/s][codecarbon INFO @ 23:37:18] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:37:18] Energy consumed for all GPUs : 0.001093 kWh. Total GPU Power : 63.544426387382124 W
[codecarbon INFO @ 23:37:18] Energy consumed for all CPUs : 0.000709 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:37:18] 0.001881 kWh of electricity used since the beginning.
Epoch 2/10 - Training:  59%|█████▉    | 371/625 [00:24<00:19, 13.19it/s][codecarbon INFO @ 23:37:33] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:37:33] Energy consumed for all GPUs : 0.001378 kWh. Total GPU Power : 68.39738994311855 W
[codecarbon INFO @ 23:37:33] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:37:33] 0.002363 kWh of electricity used since the beginning.
Epoch 2/10 - Training:  96%|█████████▌| 599/625 [00:39<00:01, 1

...

Epoch 10/10 - Training:   7%|▋         | 41/625 [00:02<00:35, 16.24it/s][codecarbon INFO @ 23:43:19] Energy consumed for RAM : 0.000554 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:43:19] Energy consumed for all GPUs : 0.007860 kWh. Total GPU Power : 64.16542672963475 W
[codecarbon INFO @ 23:43:19] Energy consumed for all CPUs : 0.004959 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:43:19] 0.013373 kWh of electricity used since the beginning.
Epoch 10/10 - Training:  43%|████▎     | 271/625 [00:17<00:22, 15.54it/s][codecarbon INFO @ 23:43:34] Energy consumed for RAM : 0.000574 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:43:34] Energy consumed for all GPUs : 0.008146 kWh. Total GPU Power : 68.83379672054959 W
[codecarbon INFO @ 23:43:34] Energy consumed for all CPUs : 0.005136 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:43:34] 0.013856 kWh of electricity used since the beginning.
Epoch 10/10 - Training:  79%|███████▉  | 495/625 [00:32<00:08, 

Epoch [10/10], Train Loss: 0.1609, Train Accuracy: 94.25%, Val Loss: 0.6651, Val Accuracy: 81.38%


[codecarbon INFO @ 23:44:04] Energy consumed for RAM : 0.000613 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:44:04] Energy consumed for all GPUs : 0.008710 kWh. Total GPU Power : 66.70825356055292 W
[codecarbon INFO @ 23:44:04] Energy consumed for all CPUs : 0.005490 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:44:04] 0.014813 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:44:05] Energy consumed for RAM : 0.000615 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:44:05] Energy consumed for all GPUs : 0.008734 kWh. Total GPU Power : 64.46371701192484 W
[codecarbon INFO @ 23:44:05] Energy consumed for all CPUs : 0.005506 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:44:05] 0.014855 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:44:05] 0.004416 g.CO2eq/s mean an estimation of 139.2680192326752 kg.CO2eq/year



--- Model Analysis After Pruning ---
Non-zero Parameter Count: 3574441
Model Size: 90.06 MB
Training Time: 7.67 minutes
Inference Time per Image: 0.022866 seconds
FLOPs: 0.08 GFLOPs
CO2 Emissions: 0.002061 kg


/usr/local/lib/python3.10/dist-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])
